<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/curiousity/curiousity_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### setup

In [ ]:
# # https://github.com/kimhc6028/pytorch-noreward-rl
# https://stackoverflow.com/questions/67808779/running-gym-atari-in-google-colab
%pip install -U gym
%pip install -U gym[atari,accept-rom-license]
# !pip install gym[box2d]
# import gym

!pip install gym-super-mario-bros nes-py
# https://github.com/Kautenja/gym-super-mario-bros
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT
# env = gym_super_mario_bros.make('SuperMarioBros-v0')
# env = JoypadSpace(env, SIMPLE_MOVEMENT)

!pip install colabgymrender
!pip install perceiver-pytorch

import gym
class SparseEnv(gym.Wrapper): #https://alexandervandekleut.github.io/gym-wrappers/
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.total_rewards = 0
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.total_rewards += reward
        if done: return observation, self.total_rewards, done, info
        else:
            self.total_rewards = 0
            return observation, 0, done, info
    def reset(self):
        self.total_rewards = 0
        return self.env.reset()
# env = SparseEnv(gym.make("LunarLander-v2"))

class MarioSparse(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.total_score = 0
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        life = info['life']
        score = info['score']
        self.total_score += score
        # print("MarioSparse",life,score)
        # if done: return observation, self.total_rewards, done, info
        if life<2: return observation, score, True, info # lost one life, end env
        else:
            # self.total_score = 0
            return observation, score, False, info
    def reset(self):
        # self.total_score = 0
        return self.env.reset()
# env = MarioSparse(env)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

log=False
# !pip install wandb
# import wandb
# wandb.login() # 
# wandb.init(project="curiousity_simple", entity="bobdole")
# log=True


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import gym
class SparseEnv(gym.Wrapper): #https://alexandervandekleut.github.io/gym-wrappers/
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.total_rewards = 0
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.total_rewards += reward
        if done: return observation, self.total_rewards, done, info
        else:
            self.total_rewards = 0
            return observation, 0, done, info
    def reset(self):
        self.total_rewards = 0
        return self.env.reset()
# env = SparseEnv(gym.make("LunarLander-v2"))

class MarioSparse(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.total_score = 0
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        if done: return observation, reward, done, info
        life = info['life']
        score = info['score']
        self.total_score += score
        if life<2:
            print("MarioSparse: died")
            return observation, score, True, info # lost one life, end env
        else:
            # self.total_score = 0
            return observation, score, False, info
    def reset(self):
        # self.total_score = 0
        return self.env.reset()
# env = MarioSparse(env)

class MarioEarlyStop(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.env = env
        self.max_pos = 0
        self.count_step = 0
    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        if done: return observation, reward, done, info
        x_pos = info['x_pos']
        if x_pos <= self.max_pos: self.count_step += 1
        else:
            self.max_pos = x_pos
            self.count_step = 0
        if self.count_step > 500:
            print("MarioEarlyStop: early stop ", self.max_pos)
            return observation, reward, True, info # early stop
        else:
            return observation, reward, False, info
    def reset(self):
        self.max_pos = 0
        self.count_step = 0
        return self.env.reset()
# env = MarioEarlyStop(env)


#### model

In [ ]:
# model.py
# https://github.com/kimhc6028/pytorch-noreward-rl/blob/master/model.py
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class ActorCritic(torch.nn.Module):
    def __init__(self, in_shape, action_space):
        super(ActorCritic, self).__init__()
        self.in_dim = in_shape # mario (240, 256)
        self.conv = nn.Sequential( # embed pi
            nn.Conv2d(in_shape[0], 32, 3, stride=2, padding=1), nn.ELU(),
            nn.Conv2d(32, 32, 3, stride=2, padding=1), nn.ELU(),
            nn.Conv2d(32, 32, 3, stride=2, padding=1), nn.ELU(),
            nn.Conv2d(32, 32, 3, stride=2, padding=1), nn.ELU(),
            nn.Conv2d(32, 32, 3, stride=2, padding=1), nn.ELU(), # added for RuntimeError: Input batch size 2 doesn't match hidden0 batch size 1
            )
        self.lstm = nn.LSTMCell(in_shape[1], 256)
        num_outputs = action_space.n
        self.actor_linear = nn.Linear(256, num_outputs) # -> action
        self.critic_linear = nn.Linear(256, 1) # -> value

        self.inv_lstm = nn.LSTMCell(in_shape[1], 256)
        self.fwd_lstm = nn.LSTMCell(in_shape[1], 256)
        self.inv_linear = nn.Sequential( # inv learning, predict at
            nn.Linear(in_shape[1] + in_shape[1], 256), nn.ReLU(),
            nn.Linear(256, num_outputs), nn.Softmax()
            ) # cat(phi(st), phi(st+1)) -> athat
        self.fwd_linear = nn.Sequential( # predict phi st+1
            nn.Linear(in_shape[1] + num_outputs, 256), nn.ReLU(),
            nn.Linear(256, in_shape[1])
            ) # cat(phi(st), at) -> phihat(st+1)

    def forward(self, inputs, icm):
        if icm == False: #A3C
            st, (a3c_hx, a3c_cx) = inputs # [1, 210, 160, 3], ([1, 256], [1, 256])
            vec_st = self.conv(st).view(-1, self.in_dim[1])
            a3c_hx1, a3c_cx1 = self.lstm(vec_st, (a3c_hx, a3c_cx))
            critic = self.critic_linear(a3c_hx1)
            actor = self.actor_linear(a3c_hx1)
            # print("forward A3C ",critic.shape, actor.shape, a3c_hx.shape, a3c_cx.shape)
            return critic, actor, (a3c_hx1, a3c_cx1) # [1, 1], [1, 12], ([1, 256], [1, 256])

        else: #icm
            (inv_hx, inv_cx), (fwd_hx, fwd_cx), st1, at = inputs
            vec_st1 = self.conv(st1).view(-1, self.in_dim[1])
            inv_hx1, inv_cx1 = self.inv_lstm(vec_st1, (inv_hx, inv_cx)) # inv model
            fwd_hx1, fwd_cx1 = self.fwd_lstm(vec_st1, (fwd_hx, fwd_cx)) # world model

            inv_vec = torch.cat((inv_hx, vec_st1), 1) # predict at
            fwd_vec = torch.cat((fwd_hx, at), 1) # predict vec_st1
            inverse = self.inv_linear(inv_vec)
            forward = self.fwd_linear(fwd_vec)
            # print("forward icm ",vec_st1.shape, inverse.shape, forward.shape)
            return vec_st1, inverse, forward, (inv_hx1, inv_cx1), (fwd_hx1, fwd_cx1) # [1, 320], [1, 18], [1, 320], ()


#### train

In [ ]:
# train.py
# https://github.com/kimhc6028/pytorch-noreward-rl/blob/master/train.py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

def train(env, args, model, optimizer=None):
    # torch.manual_seed(seed)
    # model = ActorCritic(env.observation_space.shape, env.action_space)
    if optimizer is None:
        optimizer = torch.optim.Adam(shared_model.parameters(), lr)
    model.train()
    for x in range(num_episodes):
        # model.load_state_dict(shared_model.state_dict()) # Sync with the shared model
        hx = torch.zeros(1, 256).to(device)
        cx = torch.zeros(1, 256).to(device)
        vec_st = torch.zeros(1, 256).to(device)
        inv_latent = (torch.zeros(1, 256).to(device), torch.zeros(1, 256).to(device))
        fwd_latent = (torch.zeros(1, 256).to(device), torch.zeros(1, 256).to(device))
        values = []
        log_probs = []
        rewards = []
        entropies = []
        inverses = []
        forwards = []
        actions = []
        vec_st1s = []
        episode_length = 0

        state = env.reset()
        # state=state[:,:,0]
        state = torch.from_numpy(state.copy()).type(torch.float).to(device) # i added, change from int to float
        st1 = state.float()
        # print("#####www####",state.dtype,hx.dtype)
        while True:
            episode_length += 1
            value, logit, (hx, cx) = model((state.unsqueeze(0), (hx, cx)), icm = False)
            prob = F.softmax(logit, dim=1)
            log_prob = F.log_softmax(logit, dim=1)
            entropy = -(log_prob * prob).sum(1)
            entropies.append(entropy.cpu())
            action = prob.multinomial(1).data
            log_prob = log_prob.gather(1, action)
            oh_action = torch.zeros(1, env.action_space.n)
            oh_action[0][action.item()] = 1.0
            at = oh_action
            actions.append(oh_action)
            state, reward, done, _ = env.step(action.item())
            state = torch.from_numpy(state.copy()).type(torch.float).to(device)
            # state=state[:,:,0]
            # print("reward",reward)
            done = done or episode_length >= max_episode_length
            # reward = max(min(reward, 1), -1) #why clip rewards?
            st = st1
            st1 = state.float()          
            vec_st1, inverse, forward, inv_latent, fwd_latent = model((inv_latent, fwd_latent, st1.unsqueeze(0), at.to(device)), icm = True)            

            reward_intrinsic = eta * ((vec_st1 - forward).pow(2)).sum(1) / 2.
            #reward_intrinsic = eta * ((vec_st1 - forward).pow(2)).sum(1).sqrt() / 2.
            # print("reward_intrinsic", reward_intrinsic)
            reward_intrinsic = reward_intrinsic.item()
            # print("ep ",x,", rwd ext: ", reward, " ,rwd int: ", reward_intrinsic.item())
            reward += reward_intrinsic
            values.append(value.cpu())
            log_probs.append(log_prob.cpu())
            rewards.append(reward)
            vec_st1s.append(vec_st1.cpu())
            inverses.append(inverse.cpu())
            forwards.append(forward.cpu())
            if done:
                print(episode_length)
                episode_length = 0
                break
        R = torch.zeros(1, 1)
        if not done:
            value, _, _ = model((state.unsqueeze(0), (hx, cx)), icm = False)
            R = value.data
        values.append(R)
        policy_loss = 0
        value_loss = 0
        inverse_loss = 0
        forward_loss = 0
        gae = torch.zeros(1, 1)
        for i in reversed(range(len(rewards))):
            R = gamma * R + rewards[i]
            advantage = R - values[i]
            value_loss = value_loss + 0.5 * advantage.pow(2)
            # Generalized Advantage Estimataion
            # delta_t = rewards[i] + gamma * values[i + 1].data - values[i].data
            delta_t = torch.tensor(rewards[i]) + gamma * values[i + 1].data - values[i].data
            gae = gae * gamma * tau + delta_t
            policy_loss = policy_loss - log_probs[i] * gae - 0.01 * entropies[i]
            cross_entropy = - (actions[i] * torch.log(inverses[i] + 1e-15)).sum(1)
            inverse_loss = inverse_loss + cross_entropy
            forward_err = forwards[i] - vec_st1s[i]
            forward_loss = forward_loss + 0.5 * (forward_err.pow(2)).sum(1)
        optimizer.zero_grad()
        # ((1-beta) * inverse_loss + beta * forward_loss).backward(retain_variables=True)
        inv_loss = (1-beta) * inverse_loss + beta * forward_loss
        pol_loss = lmbda * (policy_loss + 0.5 * value_loss)
        (inv_loss + pol_loss).backward()
        # (inv_loss + 0*pol_loss).backward()
        # (((1-beta) * inverse_loss + beta * forward_loss) + lmbda * (policy_loss + 0.5 * value_loss)).backward()
        print(''.join([str(torch.argmax(a).item()) for a in actions]))
        print("inv_loss: ", inv_loss.item(), " ,pol_loss: ", pol_loss.item())
        # if log: 
        #     wandb.log({"inv_loss": inv_loss.item(), "pol_loss": pol_loss.item()})
        torch.nn.utils.clip_grad_norm(model.parameters(), 40)
        optimizer.step()


#### test

In [ ]:
# test.py
# https://github.com/kimhc6028/pytorch-noreward-rl/blob/master/test.py
import numpy as np
import torch
import torch.nn.functional as F
import time

def test(env, args, model):
    # torch.manual_seed(seed)
    # model = ActorCritic(env.observation_space.shape, env.action_space)
    # model.load_state_dict(shared_model.state_dict())
    model.eval()
    state = env.reset()
    state = torch.from_numpy(state.copy()).type(torch.float)
    reward_sum = 0
    start_time = time.time()
    actions = []
    episode_length = 0
    result = []
    cx = torch.zeros(1, 256).to(device)
    hx = torch.zeros(1, 256).to(device)
    while True:
        episode_length += 1
        # value, logit, (hx, cx) = model((state.unsqueeze(0), (hx, cx)), icm = False)
        value, logit, (hx, cx) = model((state.unsqueeze(0).to(device), (hx, cx)), icm = False)
        # action = prob.max(1)[1].data.numpy() #stupid from test
        prob = F.softmax(logit, dim=1) #from train
        action = prob.multinomial(1).data
        state, reward, done, _ = env.step(action.item())
        state = torch.from_numpy(state.copy()).type(torch.float)

        done = done or episode_length >= max_episode_length
        # print("rwd ext: ", reward)
        reward_sum += reward
        actions.append(action[0])
        if done:
            end_time = time.time()
            print("Time {}, episode reward {}, episode length {}".format(
                time.strftime("%Hh %Mm %Ss", time.gmtime(end_time - start_time)), reward_sum, episode_length))
            result.append((reward_sum, end_time - start_time))
            torch.save(model.state_dict(), 'model.pth')
            # print(''.join([str(a.item()) for a in actions]))
            print([a.item() for a in actions])
            break


#### main

In [ ]:
# main.py
# https://github.com/kimhc6028/pytorch-noreward-rl/blob/master/main.py
# import os, sys, cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import gym

lr=0.001
gamma=0.99
tau=1.00
seed=1
num_processes=4
num_steps=20
max_episode_length=500 # 10000
# env_name='PongDeterministic-v4'
# env_name='LunarLander-v2'
# env_name='MontezumaRevengeDeterministic-v4'
# env_name='MontezumaRevengeDeterministic-ram-v4'

no_shared=False
eta=0.01
beta=0.2
lmbda=0.1
outdir="output"
record='store_true'
num_episodes=10#100

torch.manual_seed(seed)
# env = gym.make(env_name)
# env = SparseEnv(env)
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, COMPLEX_MOVEMENT) # SIMPLE_MOVEMENT COMPLEX_MOVEMENT
env = MarioSparse(env)
env = MarioEarlyStop(env)
# query_environment("MountainCar-v0")

# print(env.observation_space.shape, env.action_space) # (210, 160, 3) Discrete(18)

shared_model = ActorCritic(env.observation_space.shape, env.action_space).to(device)
# shared_model.share_memory()
if no_shared:
    optimizer = None
else:
    optimizer = torch.optim.Adam(shared_model.parameters(), lr=lr)
    # optimizer.share_memory()
args=None
# train(0, args, shared_model, optimizer)

# processes = []
# import torch.multiprocessing as mp
# p = mp.Process(target=test, args=(num_processes, args, shared_model))
# p.start()
# processes.append(p)
# for rank in range(0, num_processes):
#     p = mp.Process(target=train, args=(rank, args, shared_model, optimizer))
#     p.start()
#     processes.append(p)
# for p in processes:
#     p.join()



/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:565: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


#### wwwwwwwww

In [ ]:
max_episode_length=1000 # 10000


In [ ]:
# train(env, args, shared_model)

for x in range(25):
    train(env, args, shared_model, optimizer)
test(env, args, shared_model)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


MarioSparse: died
133
0104353383243334535334311101331152413413433143331333483431039333313233103441034483343411031033341113103323133333431233331133333433433213333333103
inv_loss:  172.50146484375  ,pol_loss:  -27.920881271362305


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


MarioSparse: died
113
1382423331034531033323331333133334333313133333011333382310333333323331003343310343133333381310333143314333211113334432333
inv_loss:  131.43980407714844  ,pol_loss:  -16.968381881713867
500
338433431331033104433133333343433131352333123341133333831033341333103333383413338433343434315333302103333343133443342232033133313313131032333333510333101381023383433433333103311103103310323332343311323310333331310344333211311043333334331333323343338333813333533833333831104421033433323333310383433343133333323323133031333313108333333326423311103301403331013832333833333333313103332331353333445314133233333335333433323313313113310311323339101343341310131133383733333333543103330481344233103410834348113413333311113331334341103310361021724133
inv_loss:  594.6931762695312  ,pol_loss:  704885696.0
MarioSparse: died
123
70333313332333333313543341332353333310323334238311103333803433323331333384333143333138333233333243301034438333823238103133834333
inv_loss:  142.60997009277344  ,pol_loss:  

#### save

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')
PATH="/content/gdrive/MyDrive/curious/" # for saving to google drive
# PATH="/content" # for saving on colab only
name='model_mario_lstm.pth'
model=shared_model
# torch.save(model.state_dict(), PATH+name)

# model.load_state_dict(torch.load(PATH+name))
# shared_model=model


#### video

In [ ]:

import gym
from colabgymrender.recorder import Recorder
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT

# # env = gym.make("MontezumaRevengeDeterministic-v4")
# env = SparseEnv(env)
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, COMPLEX_MOVEMENT) # SIMPLE_MOVEMENT COMPLEX_MOVEMENT
env = MarioSparse(env)
env = Recorder(env, './video')

state = env.reset()

# model = ActorCritic(env.observation_space.shape, env.action_space)
# model.load_state_dict(shared_model.state_dict())
# model.eval()
cx = torch.zeros(1, 256)
hx = torch.zeros(1, 256)
# torch.manual_seed(6)
x=0

# [11, 0, 11, 7, 9, 10, 9, 11, 1, 1, 4, 10, 1, 5, 11, 5, 1, 4, 7, 5, 11, 11, 4, 1, 9, 0, 1, 11, 1, 1, 1, 10, 1, 1, 1, 5, 9, 9, 9, 11, 9, 10, 11, 4, 10, 1, 10, 1, 1, 9, 1, 1, 11, 9, 1, 1, 10, 10, 9, 10, 11, 11, 9, 3, 10, 9, 3, 1, 9, 0, 9, 5, 8, 9, 8, 5, 3, 1, 5, 11, 1, 11, 10, 1, 9, 5, 11, 7, 3, 4, 2, 3, 0, 3, 9, 3, 1, 11, 1, 1, 3, 3, 9, 9, 9, 9, 9, 0, 9, 9, 9, 1, 4, 4, 1, 10, 10, 9, 10, 0, 4, 1, 11, 9, 1, 4, 2, 1, 9, 4, 1, 8, 0, 9, 9, 1, 2, 5, 3, 4, 6, 4, 3, 2, 2, 5, 10, 6, 10, 10, 11, 11, 10, 9, 9, 11, 0, 9, 10, 11, 4, 4, 2, 2, 10, 9, 11, 2, 9, 4, 10, 9, 8, 1, 1, 9, 3, 1, 8, 11, 9, 9, 9, 9, 0, 7, 1, 4, 4, 11, 1, 5, 1, 4, 10, 2, 5, 1, 11, 4, 4, 4, 1, 10, 3, 1, 1, 1, 11, 11, 1, 11, 11, 1, 1, 10, 9, 1, 1, 9, 5, 10, 3, 4, 11, 2, 9, 4, 1, 9, 1, 1, 1, 9, 2, 4, 1, 1, 11, 1, 4, 2, 5, 2, 9, 1, 11, 1, 9, 11, 9, 0, 1, 1, 4, 10, 7, 7, 9, 3, 11, 10, 1, 1, 1, 10, 11, 1, 2, 10, 10, 1, 9, 3, 11, 1, 5, 3, 1, 7, 9, 1, 1, 4, 1, 9, 5, 4, 1, 5, 3, 4, 11, 1, 11, 1, 4, 1, 4, 10, 11, 1, 10, 1, 11, 10, 2, 3, 4, 1, 9, 1, 9, 5, 1, 4, 9, 5, 5, 4, 11, 1, 10, 9, 10, 0, 9, 9, 9, 1, 4, 9, 1, 1, 1, 9, 4, 7, 9, 4, 10, 9, 4, 1, 5, 4, 9, 11, 10, 9, 9, 9, 1]
# [5, 0, 9, 4, 4, 4, 5, 11, 0, 8, 1, 2, 7, 11, 10, 4, 11, 11, 4, 4, 11, 4, 9, 4, 4, 1, 4, 10, 4, 0, 6, 5, 0, 6, 10, 11, 5, 11, 9, 7, 0, 4, 9, 8, 10, 4, 10, 11, 0, 2, 6, 7, 10, 2, 11, 0, 10, 2, 10, 0, 10, 4, 8, 10, 10, 4, 4, 4, 8, 4, 11, 4, 7, 9, 4, 11, 2, 4, 4, 11, 4, 10, 4, 4, 0, 4, 2, 10, 11, 4, 4, 10, 10, 4, 2, 1, 5, 0, 9, 10, 10, 0, 0, 4, 0, 4, 4, 2, 10, 0, 11, 4, 10, 8, 4, 0, 7, 7, 8, 10, 2, 4, 4, 9, 4, 4, 10, 2, 1, 8, 2, 4, 4, 4, 4, 11, 4, 4, 10, 11, 10, 0, 4, 11, 10, 3, 2, 6, 4, 4, 7, 4, 4, 4, 1, 9, 11, 4, 8, 11, 4, 4, 2, 4, 4, 7, 4, 9, 9, 11, 4, 10, 7, 11, 8, 0, 11, 2, 9, 10, 4, 11, 0, 10, 7, 4, 9, 4, 1, 11, 4, 10, 9, 4, 3, 4, 11, 0, 4, 10, 4, 11, 10, 4, 3, 0, 11, 7, 4, 4, 0, 4, 4, 10, 0, 4, 2, 11, 4, 2, 3, 11, 2, 7, 4, 9, 4, 10, 11, 11, 11, 1, 0, 10, 4, 3, 7, 8, 2, 2, 4, 5, 4, 9, 4, 4, 0, 4, 0, 11, 0, 6, 4, 11, 11, 4, 0, 0, 0, 7, 11, 0, 10, 10, 0, 10, 4, 9, 7, 4, 6, 8, 9, 4, 9, 4, 7, 2, 4, 7, 2, 8, 0, 9, 4, 6, 4, 10, 9, 7, 4, 4, 4, 8, 9, 4, 4, 9, 4, 10, 7, 4, 9, 6, 10, 11, 0, 4, 4, 0, 4, 11, 2, 0, 0, 3, 11, 0, 10, 4, 8, 10, 10, 2, 10, 10, 4, 4, 4, 1, 4, 4, 9, 0, 1, 4, 10, 4, 0, 10, 4, 0, 4, 9, 4, 7, 10, 10, 4, 4, 1, 9, 7, 7, 10, 4, 3, 10, 9, 4, 4, 4, 9, 10, 4, 8, 1, 11, 4, 9, 4, 4, 10, 1, 0, 0, 11, 1, 4, 4, 7, 9, 11, 10, 4, 1, 3, 7, 2, 4, 11, 2, 4, 8, 8, 4, 4, 3, 4, 7, 11, 0, 11, 10, 4, 11, 5, 1, 9, 4, 1, 3, 2, 9, 2, 2, 9, 4, 4, 11, 4, 7, 0, 0, 2, 2, 8, 11, 7, 6, 10, 8, 4, 0, 4, 2, 11, 0, 4, 8, 4, 4, 2, 0, 10, 7, 4, 4, 11, 8, 8, 5, 0, 0, 0, 4, 9, 7, 10, 11, 4, 9, 0, 11, 6, 8, 7, 7, 10, 4, 10, 10, 10, 0, 11, 11, 8, 6, 4, 10, 10, 9, 4, 6, 10, 4, 4, 2, 4, 11, 4, 4, 7, 2, 4, 4, 10, 4, 7, 0]
# [7, 11, 9, 7, 5, 5, 4, 1, 4, 7, 11, 5, 7, 1, 7, 6, 10, 7, 10, 10, 4, 10, 5, 7, 10, 4, 5, 2, 6, 5, 11, 4, 6, 4, 8, 5, 10, 10, 9, 10, 2, 7, 1, 6, 4, 10, 10, 2, 4, 4, 10, 3, 9, 10, 5, 5, 9, 0, 8, 10, 4, 10, 0, 7, 1, 4, 5, 9, 4, 11, 7, 10, 2, 10, 1, 1, 10, 5, 10, 10, 2, 4, 2, 10, 2, 2, 6, 1, 3, 10, 10, 4, 4, 2, 1, 9, 1, 10, 9, 4, 10, 7, 6, 2, 11, 4, 1, 4, 5, 10, 7, 10, 0, 10, 7, 10, 4, 4, 4, 3, 1, 10, 5, 10, 10, 10, 7, 10, 5, 7, 11, 10, 4, 0, 10, 7, 6, 7, 10, 4, 3, 4, 7, 5, 1, 7, 7, 7, 10, 7, 1, 4, 11, 11, 2, 8, 11, 5, 10, 2, 1, 4, 10, 6, 4, 10, 4, 1, 6, 4, 5, 6, 10, 1, 7, 10, 1, 0, 4, 7, 10, 8, 4, 4, 10, 2, 2, 10, 4, 3, 4, 8, 5, 10, 4, 2, 6, 7, 10, 5, 5, 4, 1, 10, 11, 10, 8, 4, 11, 7, 11, 7, 5, 7, 11, 10, 2, 11, 4, 10, 2, 7, 6, 6, 4, 5, 5, 1, 10, 4, 9, 8, 10, 4, 9, 2, 10, 10, 10, 2, 2, 3, 10, 10, 1, 7, 4, 10, 6, 7, 10, 10, 3, 9, 10, 10, 0, 8, 11, 7, 5, 11, 5, 5, 4, 4, 1, 2, 9, 2, 7, 7, 4, 5, 1, 4, 11, 6, 1, 3, 3, 6, 10, 10, 0, 10, 10, 7, 1, 5, 0, 4, 5, 5, 9, 7, 4, 10, 10, 5, 7, 10, 7, 4, 11, 5, 2, 2, 5, 0, 9, 3, 9, 6, 4, 9, 9, 4, 5, 1, 4, 7, 6, 9, 7, 6, 10, 4, 4, 5, 11, 7, 5, 4, 3, 10, 10, 11, 8, 5, 10, 10, 10, 11, 9, 1, 6, 5, 5, 10, 11, 10, 10, 1, 6, 2, 0, 1, 8, 5, 7, 11, 5, 2, 4, 0, 10, 4, 11, 4, 6, 3, 4, 4, 5, 4, 5, 10, 7, 7, 5, 8, 7, 1, 0, 0, 7, 7, 1, 1, 5, 5, 9, 3, 2, 4, 2, 6, 6, 10, 5, 2, 4, 7, 4, 10, 10, 4, 10, 8, 1, 10, 7, 10, 8, 10, 3, 5, 6, 4, 2, 2, 2, 7, 10, 10, 4, 9, 7, 1, 4, 6, 4, 2, 5, 7, 10, 10, 1, 11, 4, 10, 5, 1, 4, 4, 4, 8, 5, 10, 11, 4, 5, 7, 10, 2, 10, 4, 4, 6, 3, 10, 10, 10, 7, 5, 10, 2, 10, 8, 10, 9, 10, 0, 8, 5, 2, 8, 10, 5, 5, 6, 1, 11, 10, 9, 8, 7, 8, 2, 2, 1, 4, 10, 10, 3, 11, 10, 4, 5]
# acts= [10, 2, 11, 7, 2, 1, 4, 10, 1, 10, 2, 0, 2, 1, 1, 1, 10, 7, 1, 1, 6, 4, 2, 10, 4, 6, 0, 3, 6, 10, 2, 2, 11, 7, 10, 4, 10, 9, 4, 1, 4, 4, 5, 0, 1, 6, 0, 10, 7, 5, 11, 5, 4, 6, 7, 1, 1, 8, 10, 2, 5, 5, 11, 1, 1, 10, 10, 1, 0, 5, 11, 6, 2, 1, 6, 8, 4, 10, 4, 6, 2, 1, 4, 1, 7, 6, 2, 4, 1, 2, 10, 11, 2, 10, 11, 0, 1, 1, 1, 5, 1, 2, 1, 0, 3, 2, 10, 4, 1, 8, 4, 1, 9, 11, 11, 2, 2, 4, 5, 6, 1, 4, 8, 1, 10, 6, 5, 11, 2, 10, 2, 2, 2, 4, 6, 9, 10, 2, 2, 2, 6, 1, 1, 2, 4, 10, 1, 11, 2, 1, 10, 7, 1, 1, 0, 8, 2, 2, 0, 11, 1, 6, 1, 0, 2, 1, 1, 11, 0, 7, 0, 2, 4, 5, 9, 9, 2, 1, 5, 7, 0, 9, 1, 0, 10, 0, 2, 1, 2, 11, 10, 10, 6, 3, 10, 2, 10, 1, 10, 10, 7, 10, 10, 11, 0, 1, 0, 1, 4, 11, 1, 2, 10, 7, 4, 3, 0, 2, 2, 5, 2, 4, 1, 9, 3, 10, 10, 2, 1, 7, 2, 10, 2, 5, 1, 1, 4, 10, 6, 1, 9, 10, 1, 7, 10, 0, 2, 11, 7, 1, 10, 4, 11, 2, 10, 7, 1, 11, 11, 6, 0, 6, 2, 1, 0, 4, 5, 2, 1, 1, 2, 10, 11, 10, 0, 2, 2, 0, 3, 1, 0, 10, 9, 5, 1, 6, 2, 1, 4, 10, 6, 1, 4, 0, 7, 5, 9, 2, 3, 6, 7, 6, 1, 6, 10, 8, 2, 1, 7, 1, 10, 1, 10, 7, 4, 1, 1, 1, 4, 2, 2, 4, 1, 1, 10, 2, 7, 1, 1, 10, 5, 1, 1, 7, 10, 11, 4, 1, 2, 1, 0, 10, 10, 1, 11, 2, 1, 10, 8, 9, 10, 10, 9, 2, 4, 4, 7, 10, 2, 0, 1, 1, 4, 6, 5, 11, 3, 10, 7, 10, 5, 8, 11, 0, 11, 7, 1, 10, 8, 1, 9, 10, 4, 7, 7, 7, 3, 7, 3, 2, 2, 11, 10, 10, 1, 0, 10, 10, 3, 7, 10, 9, 1, 1, 5, 6, 7, 9, 11, 10, 10, 4, 10, 8, 1, 2, 11, 4, 5, 1, 5, 6, 6, 0, 1, 3, 4, 5, 1, 8, 11, 4, 10, 10, 7, 4, 1, 10, 10, 1, 11, 11, 0, 0, 7, 11, 5, 2, 6, 5, 10, 5, 1, 6, 1, 10, 11, 11, 11, 4, 0, 4, 4, 1, 7, 6, 10, 10, 4, 2, 7, 0, 7, 6, 4, 2, 10, 1, 10, 1, 2, 6, 0, 1, 10, 11, 10, 10, 10, 6, 2, 10, 6, 4, 2, 8, 11, 10, 5, 0]
# [11, 2, 11, 5, 7, 10, 11, 11, 11, 0, 2, 4, 0, 11, 10, 1, 4, 9, 6, 2, 1, 2, 7, 7, 5, 11, 2, 11, 2, 4, 11, 7, 6, 0, 11, 4, 11, 4, 10, 1, 6, 2, 5, 0, 7, 7, 11, 2, 1, 4, 2, 10, 4, 2, 2, 1, 4, 11, 0, 3, 11, 2, 2, 0, 2, 4, 11, 2, 0, 5, 2, 2, 2, 2, 4, 10, 2, 1, 2, 0, 11, 4, 10, 11, 1, 1, 10, 2, 2, 2, 2, 2, 11, 6, 11, 11, 11, 11, 6, 11, 11, 9, 6, 11, 2, 11, 11, 1, 1, 2, 4, 7, 6, 1, 2, 0, 11, 4, 7, 5, 2, 4, 4, 1, 10, 11, 2, 3, 4, 7, 4, 2, 2, 7, 11, 11, 7, 7, 1, 11, 2, 10, 11, 2, 10, 11, 11, 10, 7, 9, 1, 1, 11, 6, 2, 9, 2, 4, 2, 9, 11, 7, 2, 2, 2, 10, 11, 1, 11, 5, 10, 2, 2, 0, 11, 7, 3, 7, 10, 6, 0, 11, 5, 4, 5, 4, 11, 11, 7, 11, 2, 11, 7, 10, 4, 6, 4, 7, 2, 5, 5, 2, 2, 2, 7, 1, 6, 8, 11, 4, 10, 11, 0, 1, 4, 4, 8, 1, 2, 10, 2, 1, 7, 11, 4, 0, 9, 8, 5, 4, 4, 2, 7, 9, 1, 0, 4, 11, 11, 6, 10, 2, 5, 2, 2, 11, 2, 8, 4, 0, 2, 1, 0, 11, 4, 4, 9, 4, 7, 6, 4, 10, 11, 0, 2, 2, 5, 10, 11, 11, 2, 1, 11, 2, 5, 1, 4, 2, 0, 7, 5, 6, 7, 2, 2, 7, 2, 8, 9, 2, 9, 4, 1, 1, 4, 11, 2, 3, 2, 11, 6, 7, 11, 6, 5, 4, 7, 6, 2, 2, 4, 4, 10, 2, 7, 2, 2, 11, 3, 2, 11, 11, 2, 11, 0, 2, 7, 11, 2, 8, 7, 11, 2, 10, 11, 6, 11, 7, 6, 7, 2, 11, 2, 7, 11, 7, 2, 4, 0, 9, 2, 11, 1, 7, 7, 7, 1, 4, 11, 11, 11, 2, 0, 0, 0, 2, 4, 2, 2, 5, 2, 7, 11, 5, 10, 4, 11, 5, 11, 4, 6, 10, 2, 7, 2, 6, 2, 7, 2, 4, 11, 2, 2, 9, 3, 11, 6, 6, 2, 4, 1, 6, 10, 10, 7, 2, 1, 5, 10, 11, 2, 2, 11, 5, 7, 2, 2, 4, 5, 1, 4, 5, 4, 7, 0, 10, 4, 4, 7, 2, 8, 5, 0, 11, 0, 4, 11, 4, 2, 5, 6, 4, 11, 11, 11, 4, 1, 2, 6, 2, 10, 4, 2, 2, 11, 2, 8, 11, 5, 6, 6, 11, 6, 2, 4, 2, 1, 11, 5, 7, 7, 2, 5, 11, 1, 11, 2, 9, 2, 2, 7, 11, 5, 11, 7, 10, 11, 2, 0, 8, 11, 4, 11, 4, 5, 10, 2, 2, 6, 10]
# [9, 4, 2, 7, 1, 8, 2, 2, 11, 2, 1, 1, 11, 4, 2, 5, 1, 11, 2, 2, 1, 9, 7, 2, 2, 8, 1, 2, 2, 6, 6, 2, 2, 6, 8, 2, 2, 8, 1, 2, 2, 10, 2, 2, 6, 10, 2, 11, 11, 2, 10, 6, 10, 2, 2, 4, 2, 1, 8, 2, 10, 8, 8, 7, 2, 2, 1, 11, 1, 9, 2, 2, 5, 2, 2, 2, 6, 2, 6, 5, 2, 2, 6, 5, 10, 2, 2, 6, 1, 9, 11, 2, 9, 2, 2, 1, 2, 10, 2, 1, 8, 2, 2, 2, 2, 10, 2, 2, 2, 0, 2, 1, 10, 2, 2, 2, 2, 1, 2, 6, 6, 1, 1, 5, 2, 2, 2, 2, 2, 1, 7, 11, 8, 2, 2, 2, 6, 2, 2, 6, 2, 6, 10, 1, 2, 4, 2, 5, 7, 6, 2, 4, 2, 2, 6, 2, 2, 2, 5, 2, 4, 11, 5, 4, 2, 0, 2, 10, 9, 2, 7, 5, 2, 2, 2, 1, 2, 7, 10, 11, 8, 10, 10, 2, 6, 10, 1, 1, 3, 2, 11, 7, 2, 10, 2, 4, 0, 1, 2, 2, 6, 7, 0, 1, 2, 6, 8, 2, 8, 2, 6, 7, 2, 11, 11, 2, 1, 8, 8, 2, 5, 2, 7, 9, 3, 2, 2, 5, 2, 2, 2, 11, 7, 5, 2, 5, 7, 8, 2, 2, 6, 7, 10, 2, 1, 10, 2, 2, 2, 2, 10, 6, 5, 2, 5, 1, 1, 2, 1, 9, 2, 2, 2, 2, 11, 2, 6, 8, 2, 6, 8, 6, 1, 11, 2, 8, 10, 1, 2, 6, 2, 1, 2, 2, 7, 1, 3, 2, 2, 8, 2, 5, 2, 0, 10, 6, 0, 5, 2, 6, 6, 0, 6, 9, 5, 5, 11, 2, 2, 2, 2, 2, 1, 6, 2, 4, 6, 1, 2, 11, 2, 7, 2, 5, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 8, 2, 2, 1, 1, 8, 2, 7, 2, 6, 5, 4, 2, 11, 2, 2, 11, 1, 2, 6, 2, 2, 8, 2, 1, 2, 0, 2, 11, 2, 2, 10, 4, 2, 1, 2, 1, 5, 2, 1, 2, 2, 7, 2, 5, 2, 6, 2, 10, 2, 8, 11, 1, 1, 2, 2, 1, 2, 1, 6, 2, 2, 10, 7, 2, 2, 10, 2, 10, 2, 2, 10, 1, 5, 6, 2, 10, 6, 2, 6, 5, 2, 10, 2, 1, 7, 5, 4, 2, 6, 2, 8, 10, 11, 2, 2, 5, 2, 2, 1, 1, 2, 2, 2, 2, 2, 7, 4, 2, 2, 2, 2, 8, 1, 2, 4, 0, 8, 2, 2, 8, 7, 4, 2, 2, 6, 8, 2, 0, 8, 1, 6, 6, 2, 2, 10, 2, 2, 10, 2, 4, 11, 7, 8, 2, 2, 7, 5, 5, 6, 5, 1, 2, 5, 10, 2, 4, 1, 2, 2, 11, 6, 0, 1]
# [2, 2, 2, 2, 2, 2, 4, 1, 1, 2, 2, 2, 2, 2, 7, 6, 2, 2, 2, 2, 2, 6, 2, 2, 5, 2, 2, 0, 2, 3, 4, 2, 2, 10, 1, 10, 2, 0, 2, 2, 2, 2, 1, 10, 11, 5, 2, 1, 0, 2, 2, 1, 2, 0, 2, 5, 2, 2, 2, 2, 2, 2, 11, 2, 8, 2, 6, 2, 2, 2, 0, 2, 0, 11, 2, 2, 2, 2, 10, 4, 8, 2, 2, 2, 2, 0, 0, 0, 11, 2, 0, 1, 2, 2, 2, 5, 7, 0, 2, 2, 2, 0, 0, 9, 4, 2, 2, 0, 7, 2, 0, 8, 0, 8, 2, 2, 2, 2, 2, 2, 11, 11, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 5, 11, 11, 1, 2, 2, 1, 0, 10, 11, 2, 2, 9, 2, 10, 2, 7, 2, 2, 11, 4, 6, 2, 1, 7, 2, 2, 2, 8, 2, 5, 0, 10, 0, 11, 7, 10, 2, 3, 1, 2, 1, 8, 6, 2, 0, 2, 2, 2, 6, 2, 2, 2, 7, 0, 7, 2, 11, 0, 1, 2, 2, 7, 0, 11, 2, 2, 2, 2, 9, 2, 2, 2, 2, 2, 7, 2, 2, 11, 2, 2, 11, 2, 2, 1, 1, 2, 4, 2, 2, 2, 0, 11, 2, 2, 5, 2, 2, 2, 2, 11, 2, 8, 2, 1, 0, 2, 9, 1, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 4, 1, 1, 2, 6, 2, 2, 2, 2, 0, 9, 2, 2, 2, 2, 0, 1, 2, 2, 11, 0, 2, 6, 2, 2, 2, 7, 2, 11, 4, 2, 2, 0, 3, 11, 2, 3, 2, 0, 0, 2, 1, 0, 2, 11, 2, 2, 1, 2, 5, 5, 2, 2, 2, 2, 2, 2, 2, 0, 11, 11, 6, 2, 2, 10, 0, 2, 1, 2, 2, 7, 11, 2, 9, 1, 0, 8, 2, 2, 1, 2, 2, 0, 2, 2, 5, 0, 2, 5, 1, 6, 2, 4, 2, 0, 2, 2, 2, 2, 11, 0, 2, 2, 2, 2, 10, 0, 2, 8, 2, 4, 7, 2, 2, 0, 2, 2, 1, 2, 2, 5, 2, 2, 7, 0, 2, 0, 1, 11, 11, 2, 6, 2, 2, 0, 8, 2, 0, 1, 4, 1, 2, 0, 0, 2, 11, 2, 2, 0, 1, 2, 11, 2, 2, 2, 0, 2, 2, 2, 11, 4, 2, 2, 0, 0, 8, 2, 11, 0, 7, 4, 2, 1, 1, 10, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 7, 5, 2, 5, 5, 5, 10, 2, 2, 2, 2, 0, 2, 6, 2, 4, 1, 2, 2, 10, 6, 2, 2, 1, 2, 0, 11, 2, 7, 0, 6, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 4, 11, 1, 0, 11, 2, 1, 8, 0, 2, 0, 5, 2, 1, 2, 2, 2, 7, 10, 2, 0]
# [2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 0, 2, 2, 2, 4, 4, 2, 0, 4, 2, 10, 2, 2, 6, 4, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 0, 2, 1, 8, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 6, 2, 2, 7, 2, 7, 2, 0, 2, 2, 0, 1, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 2, 2, 2, 9, 2, 2, 11, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 7, 1, 2, 2, 2, 2, 3, 0, 2, 2, 2, 2, 2, 1, 2, 7, 2, 4, 11, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 10, 7, 1, 7, 0, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2, 8, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 8, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 10, 2, 0, 2, 2, 2, 2, 9, 2, 11, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 11, 2, 2, 2, 2, 0, 2, 11, 4, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 0, 0, 5, 2, 2, 4, 5, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 11, 2, 2, 2, 2, 2, 7, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
# [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 5, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 10, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 7, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 7, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 11, 2, 2, 5, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 9, 2, 2, 2, 1, 2, 11, 2, 2, 2, 3, 2, 2, 2, 2, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 11, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 2, 10, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 10, 2, 2, 2, 2]
acts=[5, 2, 6, 3, 2, 3, 2, 2, 2, 2, 2, 5, 3, 2, 2, 1, 2, 2, 2, 11, 11, 2, 11, 2, 2, 2, 2, 2, 5, 11, 7, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 3, 2, 1, 2, 2, 2, 3, 2, 2, 4, 2, 2, 2, 6, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2, 3, 2, 10, 5, 2, 7, 2, 2, 2, 2, 2, 2, 2, 3, 2, 11, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 10, 2, 2, 2, 2, 2, 2, 1, 3, 2, 3, 2, 2, 2, 2, 2, 2, 8, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 8, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 4, 3, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 10, 2, 2, 2, 2, 5, 2, 2, 10, 3, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 2, 2, 7, 3, 2, 2, 2, 2, 2, 9, 2, 2, 10, 2, 2, 3, 2, 2, 0, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 7, 2, 2, 2, 7, 2, 2, 5, 2, 2, 2, 5, 2, 7, 2, 2, 2, 2, 2, 3, 2, 2, 0, 10, 8, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 3, 2, 0, 7, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 6, 8, 7, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 7, 2, 2, 2, 2, 11, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 7, 2, 5, 2, 3, 2, 2, 2, 2, 5, 2, 2, 1, 10, 11, 2, 7, 2, 2, 4, 2, 2, 2, 2, 2, 7, 11, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 7, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 11, 2, 7, 2, 1, 2, 5, 6, 2, 6, 2, 2, 2, 2, 0, 2, 2, 5, 2, 0, 1, 2, 2, 2, 2, 2, 4, 2, 11, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 5, 6, 2, 2, 2, 5, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 2, 3, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 10, 2, 7, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 10, 2, 2, 2, 2, 3, 1, 2, 2, 2, 10, 2, 6, 8, 2, 2, 3, 2, 2, 2, 2, 1, 2, 8, 2, 3, 2, 1, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 6, 2, 7, 3, 7, 2, 2, 2, 7, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 7, 2, 2, 7, 7, 2, 2, 2, 5, 2, 10, 2, 1, 2, 2, 2, 9, 2, 2, 2, 2, 3, 2, 10, 0, 2, 6, 3, 3, 2, 3, 2, 2, 10, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 3, 2, 6, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 3, 2, 1, 2, 1, 11, 6, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 7, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 5, 5, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 0, 7, 2, 7, 2, 2, 2, 2, 6, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 7, 2, 2, 2, 7, 1, 2, 2, 2, 2, 2, 0, 10, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 2, 2, 2, 2, 2, 0, 1, 7, 2, 7, 11, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 7, 2, 2, 2, 3, 2, 2, 2, 2, 2, 7, 2, 2, 7, 3, 3, 3, 2, 2, 2, 2, 2, 7, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 10, 3, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 2, 2, 7, 2, 1]


while True:
    # state = torch.from_numpy(state.copy()).type(torch.float)
    # value, logit, (hx, cx) = model((state.unsqueeze(0), (hx, cx)), icm = False)
    # prob = F.softmax(logit, dim=1) #from train
    # action = prob.multinomial(1).data
    # state, reward, done, _ = env.step(action.item())
    try:
        action=int(acts[x])
    except:
        action = 10
    # # print("action",action)
    # # action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    x+=1
    if done: break
env.play()
print(x)



8020
